In [ ]:
import numpy as np

from neurodsp.sim import sim_variable_oscillation, sim_powerlaw
from neurodsp.utils.norm import normalize_sig
from neurodsp.spectral import compute_spectrum, trim_spectrum
from neurodsp.plts import plot_time_series

from fooof import FOOOF
from fooof.sim.gen import gen_periodic, gen_aperiodic

from ndspflow.motif import Motif

### Simulate Oscillations

Asymmetrical sine waves are with rise-decay symmetries of .25, .5, and .75, each at a different frequency (8, 10, or 12 hz), are simulated below.

The resulting oscillations are then combined with a powerlaw signal, to produce a 1/f slope in frequency space.

In [ ]:
# Simulate different rdsyms at 8, 10, and 12 hz
fs = 1000

sig_osc = sim_variable_oscillation(None, fs, freqs=np.repeat([8, 10, 12], 50), cycle='asine',
                                   rdsym=np.repeat([.25, .5, .75], 50))

sig_pl = sim_powerlaw(1, len(sig_osc), exponent=-2)

sig = normalize_sig(((.8 * sig_osc) + (1 * sig_pl)), mean=0, variance=1)

# Plot
times = np.arange(0, len(sig)/fs, 1/fs)
plot_time_series(times, sig)

### SpecParam
Next, the spectrum of the simulated timeseries is taken. The spectrum is then parameterized, using default, liberal settings. Later these settings will be updated base on a waveform shape analysis.

In [ ]:
# Get spectrum
freqs, powers = compute_spectrum(sig, fs, f_range=(0, 100))
freqs, powers = trim_spectrum(freqs, powers, (1, 100))
# SpecParam
fm = FOOOF()

fm.fit(freqs, powers, freq_range=(1, 100))

fm.plot(plot_peaks='shade')

### Motif

Waveform motifs are found. The three asymmetrical waves are identified in a single ~10hz oscillation peak, waveforms are untangled via k-means clustering.

Next, the additional peaks (harmonics due to asymmetry), will be removed from the spectrum. This will allows finer tuning of the oscillatory spectral peaks, while preserving an accuracte estimate of the 1/f slope.

In [ ]:
motif = Motif(min_clust_score=0.1)
motif.fit(fm, sig, fs)
motif.plot(plot_fm_kwargs={'log_freqs': True})

### Clean Spectrum
The harmonics in original spectrum will now be removed.

In [ ]:
# Determine which peaks have associated waveforms
rm_idxs = np.array([np.isnan(m.sigs).any() for m in motif.results])

# Powers in loglog
power_spectrum = fm.power_spectrum

# Remove the aperiodic fit
ap_fit = gen_aperiodic(fm.freqs, fm.aperiodic_params_)

powers_clean = power_spectrum.copy()
powers_clean -= ap_fit

for idx, peak in enumerate(fm.gaussian_params_[rm_idxs]):
    powers_clean -= gen_periodic(fm.freqs, peak)

powers_clean += ap_fit

powers_clean = 10 ** powers_clean

### Fit Distinct Oscillations

Three oscillations exists at 8, 10, and 12 hz. We will now use the cleaned spectrum and update specparam settings to fit the three oscillations separately.

Is there a way to automatically find these parameters (i.e. ml + gridsearch)?

In [ ]:
cf, bw, _ = fm.get_params('peak_params')[~rm_idxs][0]

fm_clean = FOOOF(peak_width_limits=(0, 1.6), min_peak_height=1)

fm_clean.fit(freqs, powers_clean, freq_range=(1, 100))

fm_clean.plot(plot_peaks='shade')